In [ ]:
import bbknn
import scanpy as sc

In [ ]:
dir_path = "/home/krushna/Documents/Data_integration/SCRNA_Datasets/All_h5ad/"
def load_data(dataset,batch):
    adata =sc.read_h5ad(dir_path+dataset+'.h5ad')
    sc.pp.filter_genes(adata, min_counts=3)
    adata.layers["counts"] = adata.X.copy()
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.raw = adata
    sc.pp.highly_variable_genes(
            adata,
            flavor="seurat",
            n_top_genes=2000,
            layer="counts",
            batch_key=batch,
            subset=True
    )
    return adata
    
batch_key_dic = {'Immune_Human' : 'batch',
                 'Immune_human_mouse' : 'batch',
                 'Lung' : 'batch',
                 'Mouse_brain' : 'batch',
                 'Pancreas' : 'tech',
                 'Simulation1' : 'Batch',
                 'Simulation2' : 'Batch'}
cell_type_key_dic = {'Immune_Human' : 'final_annotation',
                 'Immune_human_mouse' : 'final_annotation',
                 'Lung' : 'cell_type',
                 'Mouse_brain' : 'cell_type',
                 'Pancreas' : 'celltype',
                 'Simulation1' : 'Group',
                 'Simulation2' : 'Group'}  

In [ ]:
dataset = 'Lung_removed_top_cells'
batch = 'batch'
cell_type = 'cell_type'
adata = load_data(dataset,batch)

In [ ]:
sc.pp.pca(adata)

In [ ]:
adata_int = bbknn.bbknn(adata, batch_key=batch, copy=True)

In [ ]:
adata_int.write('bbknn-Lung_removed_top_cells.h5ad')

In [ ]:
adata_int = sc.read_h5ad("bbknn-Lung_removed_top_cells.h5ad")

In [ ]:
import scIB 
#Trajectory is asking precomputed sudo time point
results,ilisi_all,clisi_all,kbet_all  =   scIB.metrics.metrics(
        adata,
        adata_int,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=True,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=True,
        embed= 'X_pca',
        si_metric='euclidean',
        pcr_=True,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=True,  # backwards compatibility
        isolated_labels_f1_=False,
        isolated_labels_asw_=False,
        n_isolated=None,
        graph_conn_=True,
        kBET_=True,
        kBET_sub=0.5,
        lisi_graph_=True,
        lisi_raw=True,
        trajectory_=False,
        type_='knn',
        verbose=False,
)

In [ ]:
results

In [ ]:
import numpy as np
np.savetxt(dataset+"_ilisi.csv", ilisi_all, delimiter=",")
np.savetxt(dataset+"_clisi.csv", clisi_all, delimiter=",")
np.savetxt(dataset+"_kbet_all.csv",np.concatenate([np.array(val).reshape(1,-1) for val in kbet_all],axis = 0), delimiter=',')

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi


latent_matrix = adata_int.X 
labels = np.array(adata.obs[cell_type] )
K = np.size(np.unique(labels))
kmeans = KMeans(n_clusters=K, random_state=0).fit(latent_matrix)
y_pred = kmeans.labels_

print('Computing NMI ...')
NMI = nmi(labels.flatten(), y_pred.flatten())
print('NMI = {}'.format(NMI))

In [ ]:
sc.tl.umap(adata_int)
sc.pl.umap(adata_int, color=cell_type, frameon=False)
sc.pl.umap(adata_int, color=batch, frameon=False)
sc.pl.umap(adata_int, color='cluster', frameon=False)

In [ ]:
adata_int.obs['temp'] =  y_pred
adata_int.obs['temp'] = adata_int.obs['temp'].astype(str)
sc.pl.umap(adata_int, color='temp', frameon=False)